In [19]:
import os


In [20]:
%pwd

'c:\\Users\\Swarnendu\\Desktop\\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\\research'

In [21]:
import sys
sys.path.append('C:/Users/Swarnendu/Desktop/End-to-end-Chest-Cancer-Classification-using-MLflow-DVC/src')
print(sys.executable)

c:\Users\Swarnendu\anaconda3\envs\cancer\python.exe


In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [23]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [24]:
class ConfigurationManager:
    def __init__(self, config_filepath: Path, params_filepath: Path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = Path(r'C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\research\extracted_files\Chest-CT-Scan-data')
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [25]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from pathlib import Path

from PIL import Image
import scipy

In [26]:
from pathlib import Path
import tensorflow as tf

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        # Load the base model from the specified path
        base_model_path = Path(r'C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\artifacts\prepare_base_model\base_model.h5')
        base_model = tf.keras.models.load_model(base_model_path)

        # Add new layers to the model
        x = base_model.output
        x = tf.keras.layers.Flatten()(x)  # Flatten the output of the base model if necessary
        x = tf.keras.layers.Dense(2, activation='softmax')(x)  # For binary classification; use Dense(1, activation='sigmoid') if needed

        # Create a new model
        self.model = tf.keras.Model(inputs=base_model.input, outputs=x)

        # Compile the model
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.BinaryCrossentropy(),  # Use SparseCategoricalCrossentropy for multiclass classification
            metrics=['accuracy']
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [27]:
from pathlib import Path  # Make sure to import Path

try:
    # Convert file paths to Path objects
    config = ConfigurationManager(
        config_filepath=Path(r'C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\config\config.yaml'),  # Convert to Path
        params_filepath=Path(r'C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\params.yaml')   # Convert to Path
    )
    
    # Get the training configuration
    training_config = config.get_training_config()
    
    # Initialize the Training class
    training = Training(config=training_config)
    
    # Load the base model
    training.get_base_model()
    
    # Prepare the training and validation data generators
    training.train_valid_generator()
    
    # Train the model
    training.train()

except FileNotFoundError as fnf_error:
    print(f"File not found: {fnf_error}")
except Exception as e:
    print(f"An error occurred: {e}")


[2024-08-15 21:01:09,194: INFO: common: yaml file: C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\config\config.yaml loaded successfully]
[2024-08-15 21:01:09,202: INFO: common: yaml file: C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\params.yaml loaded successfully]
[2024-08-15 21:01:09,208: INFO: common: created directory at: artifacts]
[2024-08-15 21:01:09,211: INFO: common: created directory at: artifacts\training]
[2024-08-15 21:01:10,482: WARNING: hdf5_format: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
17/17 [==============================] - 325s 20s/step - loss: 0.8793 - accuracy: 0.5019 - val_loss: 0.7156 - val_accuracy: 0.3906


c:\Users\Swarnendu\anaconda3\envs\cancer\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
